In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Dict, Optional

In [3]:
# 1. 우리가 만든 툴킷 가져오기
from dolmary_schedule_toolkit import DolmaryScheduleToolkit

In [4]:
toolkits = DolmaryScheduleToolkit()
tools = toolkits.get_tools()
tools

[AddToDoTool(), ViewToDoTool(), DeleteToDoTool()]

In [7]:
# 2. 모델설정
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature=0
)

# 3. 프롬프트 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 돌마리 스케쥴을 관리하는 AI 비서야. 주어지는 도구를 잘 활용해서 사용자의 요청에 답하도록 해"),
        ("placeholder", "{history}"),
        ("human", "{question}"),
        ("placeholder", "{agent_scratchpad}")
    ]
)

# 4. agent 생성
agent = create_openai_tools_agent(
    model,
    tools,
    prompt
)
executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [8]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 2. 대화 내용 저장소 만들기
stores : Dict[str, InMemoryChatMessageHistory] = {}
def get_store(session_id : str):
    if session_id not in stores:
        stores[session_id] = InMemoryChatMessageHistory()
    return stores[session_id]

agent_history = RunnableWithMessageHistory(
    executor,
    lambda session_id: get_store(session_id),
    input_messages_key="question",
    history_messages_key="history"
)

In [14]:
cfg = {"configurable" : {"session_id" : "user-123"}}
agent_history.invoke({
    "question" : "전체 일정 보여줘"
    }, config = cfg
    )



> Entering new AgentExecutor chain...

Invoking: `view_todos` with `{}`


할일 목록은 : 12시 30분에 돌마리 파워댄스
입니다.현재 돌머리 스케줄은 12시 30분에 돌마리 파워댄스 일정이 있습니다. 다른 일정 관리가 필요하시면 말씀해 주세요!

> Finished chain.


{'question': '전체 일정 보여줘',
 'history': [HumanMessage(content='돌머리 스케줄에서 파워댄스 삭제해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='파워댄스는 현재 돌머리 스케줄에 없습니다. 다른 요청 있으신가요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='12시 30분에 돌마리 파워댄스 일정 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='12시 30분에 돌마리 파워댄스 일정이 추가되었습니다. 다른 일정 관리도 도와드릴까요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1시에 돌마리 축구 일정 추가해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1시에 돌마리 축구 일정이 추가되었습니다. 더 추가하거나 확인할 일정 있으신가요?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='전체 일정을 보여줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='현재 돌머리 스케줄은 다음과 같습니다:\n- 12시 30분에 돌마리 파워댄스\n- 1시 돌마리 축구\n\n다른 일정 관리가 필요하시면 말씀해 주세요!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='축구 일정 삭제해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='1시 돌마리 축구 일정

In [ ]:
from dolmary_schedule_toolkit import DolmaryScheduleToolkit